In [2]:
import numpy as np
import pandas as pd
from roar_py_carla.carla_agents.navigation.global_route_planner import GlobalRoutePlanner

In [9]:
dat = pd.read_csv("waypoints_new.txt", header=None)

In [10]:
print(dat)

               0           1
0    -283.799988  391.699982
1    -284.649414  393.758057
2    -289.493134  407.954468
3    -290.138977  409.847321
4    -290.784790  411.740173
...          ...         ...
2768 -280.714661  381.370667
2769 -281.256531  383.295868
2770 -281.824463  385.213501
2771 -282.418396  387.123260
2772 -283.038177  389.024780

[2773 rows x 2 columns]


In [16]:
dat_np = np.asarray(dat)
print(dat_np)

[[-283.79998779  391.69998169]
 [-284.64941406  393.75805664]
 [-289.49313354  407.95446777]
 ...
 [-281.82446289  385.21350098]
 [-282.418396    387.1232605 ]
 [-283.03817749  389.02478027]]


In [15]:
dat_np
print(dat_np.shape)
dat_np = dat_np[:,[0,2,1]]
dat_np_sparse = dat_np[::50,:]
print(dat_np_sparse.shape)

(2773, 2)


IndexError: index 2 is out of bounds for axis 1 with size 2

In [14]:
dat_np_sparse

array([[ -283.79998779,   391.69998169],
       [ -318.99038696,   499.10308838],
       [ -326.10552979,   598.36688232],
       [ -303.95117188,   695.20672607],
       [ -246.37416077,   776.7477417 ],
       [ -168.80148315,   839.03460693],
       [  -72.14686584,   870.10272217],
       [   29.24804115,   889.32128906],
       [  131.96983337,   899.65863037],
       [  234.97224426,   908.3347168 ],
       [  335.88815308,   923.16265869],
       [  402.26571655,   992.04974365],
       [  496.09024048,  1026.64648438],
       [  589.96618652,  1061.10266113],
       [  688.94171143,  1078.09069824],
       [  742.60821533,   995.34661865],
       [  757.78344727,   893.03460693],
       [  769.40740967,   788.75390625],
       [  723.30834961,   709.42022705],
       [  630.17938232,   657.4005127 ],
       [  542.42199707,   604.81500244],
       [  459.56112671,   549.0559082 ],
       [  384.54058838,   482.99816895],
       [  311.83071899,   414.34857178],
       [  240.26

In [4]:
import carla
import roar_py_carla
import roar_py_interface
import networkx as nx

carla_client = carla.Client('localhost', 2000)
carla_client.set_timeout(5.0)
roar_py_instance = roar_py_carla.RoarPyCarlaInstance(carla_client)
roar_py_world = roar_py_instance.world
grp = GlobalRoutePlanner(roar_py_world._native_carla_map, 2.0)
native_ws = []


for i in range(len(dat_np_sparse)):
    curr_start = dat_np_sparse[i]
    curr_end = dat_np_sparse[(i+1)%len(dat_np_sparse)]
    loc_1 = carla.Location(curr_start[0], curr_start[1], curr_start[2])
    loc_2 = carla.Location(curr_end[0], curr_end[1], curr_end[2])
    try:
        native_ws += grp.trace_route(loc_1, loc_2)
    except nx.NetworkXNoPath:
        continue

NameError: name 'dat_np_sparse' is not defined

In [16]:
real_ws = []
for native_ww in native_ws:
    w = native_ww[0]
    transform_w = roar_py_carla.transform_from_carla(w.transform)
    transform_w[1][2] += np.pi/2
    real_w = roar_py_interface.RoarPyWaypoint(
        transform_w[0],
        transform_w[1],
        w.lane_width
    )
    real_ws.append(real_w)

In [17]:
np.savez_compressed("final_major_map_waypoints.npz", **roar_py_interface.RoarPyWaypoint.save_waypoint_list(real_ws))

In [28]:
from roar_py_interface import RoarPyWaypoint
from typing import List, Tuple
from roar_py_carla.utils import *
import carla

def generate_waypoints(locations : List[np.ndarray], distance_between_waypoints : float, lane_width : float):
    locations = [location_from_carla(carla.Location(x=location[0], y=location[1], z=location[2])) for location in locations]
    waypoint_locations = [locations[0]]
    waypoint_rotations = []
    last_location_idx = 0
    for i in range(1,len(locations)):
        next_location = locations[i]
        dist_to_last_location = np.linalg.norm(next_location - locations[last_location_idx])
        if dist_to_last_location > distance_between_waypoints:
            waypoint_locations.append(next_location)
            waypoint_rotations.append(np.arctan2(next_location[1] - locations[last_location_idx][1], next_location[0] - locations[last_location_idx][0]))
            last_location_idx = i
    waypoint_rotations.append(np.arctan2(locations[0][1] - waypoint_locations[-1][1], locations[0][0] - waypoint_locations[-1][0]))
    waypoints = []
    for i in range(len(waypoint_locations)):
        waypoints.append(RoarPyWaypoint(
            waypoint_locations[i],
            np.array([0,0,waypoint_rotations[i]]),
            lane_width
        ))
    return waypoints

waypoints = generate_waypoints(dat_np, 2, 3.5)
print("final length",len(waypoints))

np.savez_compressed("final_major_map_waypoints.npz", **RoarPyWaypoint.save_waypoint_list(waypoints))
print(waypoints[0].location)

final length 6942
[-4372.60009766 -2599.19995117   104.74610138]


In [26]:

def generate_spawn_points(waypoints : List[RoarPyWaypoint]) -> Tuple[np.ndarray, np.ndarray]:
    spawn_point_locations = []
    spawn_rotations = []
    for spawn_point in waypoints[::len(waypoints)//10]:
        spawn_point_locations.append(spawn_point.location)
        spawn_rotations.append(spawn_point.roll_pitch_yaw)
    return np.stack(spawn_point_locations, axis=0), np.stack(spawn_rotations, axis=0)

spawn_point_locations, spawn_rotations = generate_spawn_points(waypoints)
print(spawn_point_locations.shape)
np.savez_compressed("final_major_map_spawn_points.npz", locations=spawn_point_locations, rotations=spawn_rotations)

(11, 3)
